In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv

# Setup WebDriver
driver = webdriver.Chrome()
driver.get("https://www.dentons.com/en/our-professionals")
wait = WebDriverWait(driver, 10)
access_rates = 5
# Open CSV file
with open('dentons_profiles.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Name', 'Profile URL', 'Job Title', 'Office', 'Email Available', 'Phone Number'])

    scraped_urls = set()

    while True:
        # Wait for person elements to load
        wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.person-tab-wrapper")))
        persons = driver.find_elements(By.CSS_SELECTOR, "div.person-tab-wrapper")

        for index, person in enumerate(persons):
            try:
                profile_element = person.find_element(By.CSS_SELECTOR, "a.name")
                profile_url = profile_element.get_attribute("href")
                if profile_url in scraped_urls:
                    continue
                scraped_urls.add(profile_url)
                name = profile_element.find_element(By.CSS_SELECTOR, "h4").text.strip()
            except:
                name = ""
                profile_url = ""

            try:
                job_title = person.find_element(By.CSS_SELECTOR, ".info-position p").text.strip()
            except:
                job_title = ""

            try:
                office = person.find_elements(By.CSS_SELECTOR, ".info-position p")[1].text.strip()
            except:
                office = ""

            try:
                email_element = person.find_element(By.CSS_SELECTOR, "a.email-me")
                email_available = 'Yes' if email_element.is_displayed() else 'No'
            except:
                email_available = 'No'

            # Expand section to get phone number
            phone_number = ""
            try:
                expand_button = person.find_element(By.CSS_SELECTOR, "div.expand-plus")
                driver.execute_script("arguments[0].click();", expand_button)
                # Wait for the expanded content to load
                expand_id = f"expandHTML_{index}"
                wait.until(EC.presence_of_element_located((By.ID, expand_id)))
                expand_container = driver.find_element(By.ID, expand_id)
                phone_element = expand_container.find_element(By.CSS_SELECTOR, "a.phone")
                phone_number = phone_element.text.strip()
            except:
                phone_number = ""

            writer.writerow([name, profile_url, job_title, office, email_available, phone_number])
            print(f"📞 {name} | {job_title} | {profile_url} | {office} | {phone_number}")

        # Try clicking "Load more professionals" button
        try:
            load_more_button = driver.find_element(By.ID, "get-more")
            if load_more_button.is_displayed() and load_more_button.is_enabled():
                driver.execute_script("arguments[0].click();", load_more_button)
                print("🔄 Clicked 'Load more professionals' button")
                time.sleep(access_rates)  # Wait for new profiles to load
            else:
                print("❌ 'Load more' button not visible or disabled")
                break
        except:
            print("✅ No more 'Load more' button found")
            break

# Cleanup
driver.quit()
print("🎉 Scraping completed.")
